In [ ]:
import os
import numpy as np
import pandas as pd

class AlterationNumericalAttack:
  
    def __init__(self, dataset, watermarked_data_path, random_seed, attack_proportions=None, p=20, perturbed_attribute = 'MEDV'):
      
        self.watermarked_data_path = watermarked_data_path
        self.attack_proportions = attack_proportions if attack_proportions is not None else [0.2, 0.4, 0.6, 0.8, 1.0]
        self.p = p
        self.random_seed = random_seed
        self.perturbed_attribute = perturbed_attribute
        self.dataset = dataset
        np.random.seed(self.random_seed)
        
        self.loaded_results = np.load(self.watermarked_data_path, allow_pickle=True).item()
        self.watermarked_data = self.loaded_results['watermarked_data']
        
    def apply_attack(self, proportion):
       
        temp = self.watermarked_data.copy()
        indices = np.random.choice(len(temp), size=int(proportion * len(temp)), replace=False)
        perturb_values = np.random.uniform(-self.p, self.p, size=len(indices))  # 扰动值
        temp.loc[indices, self.perturbed_attribute] += perturb_values
        attack_file_path = f"datasets/attack/alteration/{self.dataset}/alteration_{proportion}.csv"
        temp.to_csv(attack_file_path, index=False)
        print(f"Alteration attack applied with proportion {proportion}. Modified data saved to {attack_file_path}")

    def execute_attack(self):
        for proportion in self.attack_proportions:
            self.apply_attack(proportion)

if __name__ == "__main__":
    dataset = "housing"
    seed = 10000
    attack = AlterationNumericalAttack(dataset, f"{dataset}-{seed}.npy", seed)
    attack.execute_attack()



In [ ]:
import os
import numpy as np
import pandas as pd

class InsertionNumericalAttack:
    def __init__(self, dataset, watermarked_data_path, seed, attack_proportions=None):
       
        self.dataset = dataset
        self.seed = seed
        self.watermarked_data_path = watermarked_data_path
        self.attack_proportions = attack_proportions or [0.2, 0.4, 0.6, 0.8, 1.0]
        
        np.random.seed(self.seed)
        
        self.loaded_results = np.load(f"{self.watermarked_data_path}", allow_pickle=True).item()
        self.watermarked_data = self.loaded_results['watermarked_data']
        
        self.medv_max = self.watermarked_data['MEDV'].max()
        self.medv_min = self.watermarked_data['MEDV'].min()

    def insert_tuples(self, temp, proportion):
    
        num_insertions = int(proportion * len(temp))
        sampled_rows = pd.DataFrame(columns=temp.columns)
        
        for column in temp.columns:
            if column == 'MEDV':
                sampled_rows[column] = np.random.uniform(self.medv_min, self.medv_max, size=num_insertions)
            else:
                sampled_rows[column] = temp[column].sample(n=num_insertions, replace=True).reset_index(drop=True)
        
        insertion_indices = np.random.choice(len(temp) + num_insertions, size=num_insertions, replace=False)
        
        expanded_temp = pd.DataFrame(index=range(len(temp) + num_insertions), columns=temp.columns)
        original_indices = np.setdiff1d(np.arange(len(temp) + num_insertions), insertion_indices)
        
        expanded_temp.iloc[original_indices] = temp.values
        expanded_temp.iloc[insertion_indices] = sampled_rows.values
        
        return expanded_temp

    def execute_attack(self):

        for proportion in self.attack_proportions:
            temp = self.watermarked_data.copy()
            temp = self.insert_tuples(temp, proportion)
            output_file = f"datasets/attack/insertion/{self.dataset}/insertion_{proportion}.csv"
            temp.to_csv(output_file, index=False)
            print(f"Attack with proportion {proportion} saved to {output_file}")
    
if __name__ == "__main__":
    dataset = "housing"
    seed = 10000
    attack = InsertionNumericalAttack(dataset, f"{dataset}-{seed}.npy", seed)
    attack.execute_attack()


In [ ]:
import os
import numpy as np
import pandas as pd

class DeletionNumericalAttack:
    def __init__(self, dataset, watermarked_data_path, seed, attack_proportions=None):
       
        self.dataset = dataset
        self.seed = seed
        self.attack_proportions = attack_proportions or [0.2, 0.4, 0.6, 0.8, 1.0]
        self.watermarked_data_path = watermarked_data_path
        
        np.random.seed(self.seed)
        
        self.loaded_results = np.load(f"{self.dataset}-{self.seed}.npy", allow_pickle=True).item()
        self.watermarked_data = self.loaded_results['watermarked_data']
        
    def execute_deletion(self, proportion):
       
        temp = self.watermarked_data.copy()
        indices = np.random.choice(len(temp), size=int(proportion * len(temp)), replace=False)
        temp = temp.drop(indices)
        
        return temp

    def execute_attack(self):
      
        for proportion in self.attack_proportions:
            temp = self.execute_deletion(proportion)
            output_file = f"datasets/attack/deletion/{self.dataset}/deletion_{proportion}.csv"
            temp.to_csv(output_file, index=False)
            print(f"Deletion attack with proportion {proportion} saved to {output_file}")

if __name__ == "__main__":
    dataset = "housing"
    seed = 10000
    attack = DeletionNumericalAttack(dataset, f"{dataset}-{seed}.npy", seed)
    attack.execute_attack()
